<a href="https://colab.research.google.com/github/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/Algoritmo_Genetico_para_el_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install requests
import urllib.request

!pip install tsplib95
import tsplib95

In [0]:
import random   #Libreria para generar numeros y listas aleatorias
import copy     #Permite hacer copias de objetos en python: listas, diccionarios,...

In [0]:
#Librerias y carga del problema

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/usage.html
  # https://tsplib95.readthedocs.io/modules.html#module-tsplib95.models

#Matriz de distancias
file = "swiss42.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

#Objeto de tsplib95 para nuestro problema problema
problem = tsplib95.load_problem(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

#Coordenadas(si estan disponibles en el ficher)
problem.get_display(1)

#Distancia 
problem.wfunc(1, 2)

#Matriz de distancias
# [ [problem.wfunc(i, j)   for i in  list(problem.get_nodes()) ]  for j in  list(problem.get_nodes()) ]  

34

In [0]:
#Funciones de la Actividad Guiada 3
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [0]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  Nodos = list(problem.get_nodes())
  poblacion = generar_poblacion(Nodos,N)
  
  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)
 
  
  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :
    
    #Cruce de la poblacion(incluye mutación)
    poblacion = Cruzar(poblacion,mutacion, problem)
  
    #Seleccionamos la población
    poblacion = Seleccionar(problem,poblacion, N, elitismo)
      
    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)
  
    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon distancia " , mejor_distancia, "\n")
 
    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1
    
  return mejor_solucion
  
#1º intento  :2113
#sol = algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=200)

#2º intento. Aumentamos la poblacion:1654
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=200)

#3º intento. Aumentamos las generaciones:2055
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.1,generaciones=250)

#4º intento. Aumentamos el elitismo: 1728
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.15,elitismo=.40,generaciones=250)

#5º intento. Aumentamos la mutacion : 1653
#sol = algoritmo_genetico(problem=problem,N=500,mutacion=.3,elitismo=.40,generaciones=250)

In [0]:
#Genera una poblacion inicial de soluciones de tamaño N
def generar_poblacion(Nodos,N):
  return [crear_solucion(Nodos) for _ in range(N) ]


#Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion, problem):
  mejor_solucion = []
  mejor_distancia = 10e100
  for p in poblacion:
    #print("solucion:", p)
    distancia_referencia = distancia_total(p, problem)
    if distancia_referencia < mejor_distancia:
      mejor_solucion = p
      mejor_distancia = distancia_referencia
  return mejor_solucion, mejor_distancia   

#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.
def Cruzar(poblacion,mutacion, problem):
  #Definimos en una variable la copia de la población para ir eliminando los padres seleccionados
  poblacion_copia = copy.deepcopy(poblacion)

  #Definimos en una variable la copia de la población para ir añadiendo los hijos creados
  poblacion_final = copy.deepcopy(poblacion)

  while len(poblacion_copia) > 1:  #Iteramos mientras haya padres disponibles
    #Seleccionamos dos padres
    padre1,padre2 = random.sample(poblacion_copia   ,2)
    poblacion_copia.remove(padre1)
    poblacion_copia.remove(padre2)
    poblacion_final.extend(Descendencia([padre1,padre2],problem,mutacion))
  return poblacion_final

#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte pero es posible usar otros n-puntos, uniforme, dependiendo del problema
def Descendencia(padres, problem,mutacion):
  #Se elige un punto de corte aleatorio:
  pc = random.sample( range(1,len(padres[0])),1)[0]
  hijo1 =  Factibilizar(padres[0][:pc] + padres[1][pc:] ,problem) 
  hijo2 =  Factibilizar(padres[1][:pc] + padres[0][pc:] ,problem)
  return [hijo1,hijo2,Mutar(hijo1,mutacion),Mutar(hijo2,mutacion)]

#Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(solucion,problem):
  Nodos = list(problem.get_nodes())
  nodos_desaparecidos = list(set(Nodos) - set(solucion))
  #Recorremos todos los nodos, cuando haya uno que ya esté en la lista los cambiamos por uno de la lista de nodos_desaparecidos
  for i in range(len(solucion) ):
    if solucion[i] in solucion[:i]:
      #print("\tSe repite el",solucion[i] )
      #print("\tSe cambia en la posicion ",i , " " ,solucion[i] , " por ", nodos_desaparecidos[0] )
      solucion[i] = nodos_desaparecidos.pop(0) #Cambiamos el nodo y a la vez eliminamos el nodo usado de nodos_desaparecidos
  return solucion 

#Funcion de mutación. Se eligen dos nodos y se intercambia. Se podrian añadir otros operaradores
# Se hace mutaciones mutacion% de las veces
def Mutar(solucion,mutacion):
  if random.random() < mutacion:
    sel1,sel2 = sorted(random.sample(set(solucion)-{solucion[0]},2))
    return solucion[:sel1] + [solucion[sel2]] + solucion[sel1+1:sel2] + [solucion[sel1]]  + solucion[sel2+1:] 
  else:
    return solucion[::] 

#Funcion de seleccion de la población. Recibe como parametro una poblacion y 
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro 
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)
def Seleccionar(problem,poblacion, N, elitismo):
  #Se ordena la población según el fitness(tamaño del recorrido) en una lista de elementos [distancia, solucion]
  poblacion_ordenada = sorted([ [distancia_total(solucion, problem), solucion] for solucion in poblacion ], key= lambda x:x[0] )

  #Devolvemos elitismo% y el resto se eligen aleatoriamente
  return [x[1] for x in poblacion_ordenada][:int(N*elitismo)]  + \
  random.sample([x[1] for x in poblacion_ordenada][int(N*elitismo):] , int(N*(1-elitismo))  ) 
  
     